<a href="https://colab.research.google.com/github/omorros/Fruit-vision-cnn/blob/main/use_trained_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fruit Classifier Reload, Evaluation & Prediction Notebook

This notebook provides a complete, step-by-step guide to loading, evaluating, and testing a pre-trained convolutional neural network (**CNN**) for fruit image classification.  

It is intended as a self-contained tutorial for anyone who wants to understand how to reuse a trained deep learning model, evaluate its performance, and make predictions on new images.  

### The notebook walks through the following main stages:

1. Setting up the Colab environment and verifying TensorFlow configuration  
2. Connecting to the Kaggle API and downloading the Fruits-360 dataset  
3. Loading the pre-trained CNN model (`.keras`) and label mapping (`.json`)  
4. Uploading individual images to perform classification  
5. Optionally evaluating model performance on the test dataset  
6. Saving or backing up the model for future use in other projects  

Each step is clearly explained and documented to ensure that the process can be replicated by users of all experience levels.The notebook can serve as a foundation for integrating fruit recognition functionality into larger computer vision or machine learning applications.

In [1]:
# ==============================================================
# STEP 0 — SETUP: Install Libraries & Check Environment
# ==============================================================

# Install necessary packages
!pip install -q tensorflow keras kaggle

# Import key libraries
import tensorflow as tf, platform, sys, os, random, numpy as np

# Display environment info
print("TensorFlow version:", tf.__version__)
print("Python version:", sys.version.split()[0])
print("Operating System:", platform.platform())
print("Available GPU devices:", tf.config.list_physical_devices('GPU'))

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)



TensorFlow version: 2.19.0
Python version: 3.12.12
Operating System: Linux-6.6.105+-x86_64-with-glibc2.35
Available GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# ==============================================================
# STEP 1 — KAGGLE API SETUP
# ==============================================================

# We use the Kaggle API to download the official Fruits-360 dataset.
# To do this, the user must upload their Kaggle API key (kaggle.json).
# Get it from: https://www.kaggle.com -> Profile -> Account -> "Create API Token"

from google.colab import files

print("Please upload your 'kaggle.json' API key file from Kaggle.")
uploaded = files.upload()  # upload kaggle.json

# Once uploaded, we place it in the hidden .kaggle folder
# and set proper permissions to use it securely.
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("\nKaggle API configured successfully.")


Please upload your 'kaggle.json' API key file from Kaggle.


Saving kaggle.json to kaggle.json

Kaggle API configured successfully.


In [3]:
# ==============================================================
# STEP 2 — DOWNLOAD & UNZIP FRUITS-360 DATASET
# ==============================================================

# This step downloads the Fruits-360 dataset directly from Kaggle.
# It only runs if the dataset does not already exist.

!pip install -q kaggle

if not os.path.exists("/content/fruits_dataset"):
    print("⬇Downloading Fruits-360 dataset...")
    !kaggle datasets download -d moltean/fruits -p /content
    print("Unzipping dataset, please wait...")
    !unzip -q /content/fruits.zip -d /content/fruits_dataset
    print("Dataset ready at /content/fruits_dataset/")
else:
    print("Dataset already exists — skipping download.")


⬇Downloading Fruits-360 dataset...
Dataset URL: https://www.kaggle.com/datasets/moltean/fruits
License(s): CC-BY-SA-4.0
100% 4.54G/4.54G [03:19<00:00, 30.6MB/s]
100% 4.54G/4.54G [03:19<00:00, 24.4MB/s]
Unzipping dataset, please wait...
Dataset ready at /content/fruits_dataset/


In [7]:
# ==============================================================
# STEP 3 — UPLOAD OR LOAD TRAINED MODEL & LABEL FILES
# ==============================================================

import json

print("Option A: Upload model manually from your computer")
print("Option B: Load automatically from Google Drive (if previously saved)")

try:
    choice = input("\nEnter A or B: ").strip().lower()
except:
    print("Colab detected an issue with input(). Defaulting to Option A.")
    choice = "a"

if choice == "a":
    from google.colab import files
    print("\nPlease upload both files: 'fruits_classifier.keras' and 'fruits_labels.json'")
    uploaded = files.upload()
    model = tf.keras.models.load_model("fruits_classifier.keras")
    with open("fruits_labels.json") as f:
        idx_to_class = json.load(f)
    print("Model and labels loaded from local upload.")

elif choice == "b":
    from google.colab import drive
    drive.mount('/content/drive')
    MODEL_PATH = "/content/drive/MyDrive/SmartFoodAI/fruits_classifier.keras"
    LABELS_PATH = "/content/drive/MyDrive/SmartFoodAI/fruits_labels.json"
    model = tf.keras.models.load_model(MODEL_PATH)
    with open(LABELS_PATH) as f:
        idx_to_class = json.load(f)
    print("Model and labels loaded from Google Drive.")
else:
    raise ValueError("Invalid choice. Please restart and type 'A' or 'B'.")

print(f"\nTotal fruit classes: {len(idx_to_class)}")


Option A: Upload model manually from your computer
Option B: Load automatically from Google Drive (if previously saved)

Enter A or B: a

Please upload both files: 'fruits_classifier.keras' and 'fruits_labels.json'


Saving fruits_classifier.keras to fruits_classifier (1).keras
Saving fruits_labels.json to fruits_labels (1).json
Model and labels loaded from local upload.

Total fruit classes: 225


In [8]:
# ==============================================================
# STEP 4 — UPLOAD AN IMAGE AND MAKE A PREDICTION
# ==============================================================

# Test the trained CNN on any uploaded fruit image (JPG or PNG).
# The model will predict which fruit it is and display its confidence score.

from tensorflow.keras.preprocessing import image
import numpy as np

print("Upload an image to classify (e.g., apple, banana, lemon)...")
uploaded = files.upload()
img_path = list(uploaded.keys())[0]

# Image size used in training
IMG_SIZE = (100, 100)

# Preprocess the image: resize, normalize, add batch dimension
img = image.load_img(img_path, target_size=IMG_SIZE)
x = image.img_to_array(img) / 255.0
x = np.expand_dims(x, axis=0)

# Make prediction
probs = model.predict(x)
pred_idx = int(np.argmax(probs))
pred_label = idx_to_class[str(pred_idx)] if isinstance(list(idx_to_class.keys())[0], str) else idx_to_class[pred_idx]

print("\nPrediction Result:")
print(f"Predicted fruit: {pred_label}")
print(f"Confidence: {probs[0][pred_idx]:.2f}")


📸 Upload an image to classify (e.g., apple, banana, lemon)...


Saving Screenshot 2025-10-31 175548.png to Screenshot 2025-10-31 175548.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

Prediction Result:
Predicted fruit: Cauliflower 1
Confidence: 1.00


In [9]:
# ==============================================================
# STEP 5 — EVALUATE MODEL ON TEST DATASET
# ==============================================================

# This step checks whether the test folder from Fruits-360 is available.
# If so, it evaluates your model accuracy on the test set.
# If not, it automatically skips this step.

from tensorflow.keras.preprocessing.image import ImageDataGenerator

TEST_DIR = "/content/fruits_dataset/fruits-360_100x100/fruits-360/Test"
IMG_SIZE = (100, 100)
BATCH = 32

if os.path.exists(TEST_DIR):
    print("Found test dataset — running evaluation...")

    test_datagen = ImageDataGenerator(rescale=1./255)
    test_gen = test_datagen.flow_from_directory(
        TEST_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH,
        class_mode='categorical',
        shuffle=False
    )

    test_loss, test_acc = model.evaluate(test_gen, verbose=0)
    print("\nEvaluation complete:")
    print(f"Accuracy on test set: {test_acc*100:.2f}%")
    print(f"Loss on test set: {test_loss:.4f}")

else:
    print("Test dataset not found — skipping evaluation.")
    print("You can redownload the dataset in Step 2 to enable this step.")


Found test dataset — running evaluation...
Found 39212 images belonging to 225 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Evaluation complete:
Accuracy on test set: 96.30%
Loss on test set: 0.1592


In [14]:
# ==============================================================
# STEP 6 — SAVE OR BACKUP MODEL AGAIN
# ==============================================================

# If you want to save the reloaded model again to Drive or your computer,
# this cell makes it easy to back it up.

import shutil
import os
from google.colab import drive

# Mount Drive if needed
drive.mount('/content/drive')

SAVE_DIR = "/content/drive/MyDrive/SmartFoodAI"
os.makedirs(SAVE_DIR, exist_ok=True)

# Copy model and labels
shutil.copy("fruits_classifier.keras", SAVE_DIR)
shutil.copy("fruits_labels.json", SAVE_DIR)

print("Model and labels backed up to Google Drive:", SAVE_DIR)

# Optional: Download directly to your PC
# from google.colab import files
# files.download("fruits_classifier.keras")
# files.download("fruits_labels.json")


Mounted at /content/drive
Model and labels backed up to Google Drive: /content/drive/MyDrive/SmartFoodAI
